# Grounded Segement Anything



In [ ]:
# ! python -m pip install -e segment_anything
# ! python -m pip install -e GroundingDINO
# ! pip install diffusers transformers accelerate scipy safetensors
# pip install addict
# pip install yapf
# pip install pycocotools
# pip install timm
# pip install supervision
# pip install segment_anything

In [ ]:
import sys
sys.path.append("Grounded-Segment-Anything")


In [14]:
import os
import torch
import cv2
from PIL import Image, ImageDraw
from GroundingDINO.groundingdino.util.inference import load_model, load_image, predict
from GroundingDINO.groundingdino.util import box_ops

# Config paths - À ADAPTER À VOTRE STRUCTURE
CONFIG_PATH = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
WEIGHTS_PATH = "GroundingDINO/weights/groundingdino_swint_ogc.pth"

# Créer le dossier weights si besoin
os.makedirs(os.path.dirname(WEIGHTS_PATH), exist_ok=True)

# Télécharger les poids si absent (optionnel)
if not os.path.exists(WEIGHTS_PATH):
    import requests
    print("Téléchargement des poids du modèle...")
    url = "https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth"
    r = requests.get(url, allow_redirects=True)
    with open(WEIGHTS_PATH, 'wb') as f:
        f.write(r.content)

# Charger le modèle
device = "cuda" if torch.cuda.is_available() else "cpu"
model = load_model(CONFIG_PATH, WEIGHTS_PATH, device=device)

# ... (le reste de votre code reste identique)

Téléchargement des poids du modèle...
final text_encoder_type: bert-base-uncased


In [15]:

# Dossier d'entrée
input_dir = "dataset_images"
output_dir = "gsam_output"
os.makedirs(output_dir, exist_ok=True)

# Liste des prompts et couleurs
prompts = {
    "header": "web page header section",
    "footer": "bottom of web page",
    "content": "main article or central content",
    "media": "image or video",
    "ads": "advertisement or banner",
    "sidebar": "side panel or sidebar menu"
}

colors = {
    "header": "green",
    "footer": "blue",
    "content": "orange",
    "media": "pink",
    "ads": "red",
    "sidebar": "yellow"
}

# Sélectionner 2 images
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png'))][:2]

for img_name in image_files:
    print(f"\n🖼️ Traitement de {img_name}...")
    img_path = os.path.join(input_dir, img_name)
    
    # Charger l'image
    image_source, image = load_image(img_path)
    
    # Convertir pour PIL
    pil_img = Image.fromarray(cv2.cvtColor(image_source, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)

    # Appliquer chaque prompt
    for label, prompt in prompts.items():
        boxes, logits, phrases = predict(
            model=model,
            image=image,
            caption=prompt,
            box_threshold=0.35,
            text_threshold=0.25,
            device=device
        )
        
        print(f" - {label}: {len(boxes)} boxes trouvées")

        # Convertir les boxes en coordonnées d'image
        H, W, _ = image_source.shape
        boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

        for box in boxes_xyxy:
            x1, y1, x2, y2 = map(int, box)
            draw.rectangle([x1, y1, x2, y2], outline=colors[label], width=3)
            draw.text((x1 + 5, y1 + 5), label, fill=colors[label])

    # Sauvegarde
    output_path = os.path.join(output_dir, img_name)
    pil_img.save(output_path)
    print(f"✅ Sauvegardée dans {output_path}")


🖼️ Traitement de AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_0_1743087429.jpg...


 - header: 0 boxes trouvées
 - footer: 1 boxes trouvées
 - content: 0 boxes trouvées
 - media: 1 boxes trouvées
 - ads: 1 boxes trouvées
 - sidebar: 2 boxes trouvées
✅ Sauvegardée dans gsam_output\AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_0_1743087429.jpg

🖼️ Traitement de AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_0_1743313884.jpg...
 - header: 2 boxes trouvées
 - footer: 1 boxes trouvées
 - content: 1 boxes trouvées
 - media: 1 boxes trouvées
 - ads: 1 boxes trouvées
 - sidebar: 1 boxes trouvées
✅ Sauvegardée dans gsam_output\AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_0_1743313884.jpg
